# Update - Download New OISST Data'

This is basically the babystep towards continuous integration. Use Beautifulsoup to ping the url for re-downloading the most recent OISST data. This step will then kick off subsequent `Update`  workflow steps.

In [1]:
from bs4 import BeautifulSoup
import requests
import os
import xarray as xr

### Set up the download information

In [25]:
# Update year to search for among links
update_yr = 2020
update_month = 12

# Out Destination - Cache gobals
_cache_root = "RES Data/OISST/oisst_mainstays"

# Cache Subdirectory Locations
cache_locs = {
  "cache_root" : _cache_root,
  "annual_file_cache" : _cache_root + "/oisst_annual_sst/" ,
  "regional_cache" : _cache_root + "/regional_sst_timelines/" 
}

# Set output location
cache_loc = cache_locs["annual_file_cache"]

# Root url where the yearly files are stored

# This url does  not work as it is the return for a data query. When using bs4 it scrapes the site prior to the search
#fetch_url = "https://psl.noaa.gov/cgi-bin/db_search/DBListFiles.pl?did=132&tid=89745&vid=2423"

# Use the url from Eric's Repo, thank you Eric

fetch_url = f"https://www.ncei.noaa.gov/data/sea-surface-temperature-optimum-interpolation/v2.1/access/avhrr/{update_yr}{update_month}/"

### Open the http directory listing

In [26]:
# open the http directory listing.
req = requests.get(fetch_url)

# Print error message if link does not work
if req.status_code != requests.codes.ok:
    #logger.info("{} {} {}".format(req.status_code, req.reason, req.url))
    print(f"Request Error, Reason: {req.reason}")

Set up the destination folder connection. would be weird if it did not exist, but check anyways. Going to keep all the years in one folder so that they are easier to load with xr.open_mfdataset(). 

Parse req with BS

In [27]:
 # Parse it with BS and its html parser.
soup = BeautifulSoup(req.text, 'html.parser')

In [29]:
# Find all href anchors in the html text
anchors = soup.find_all("a")
for link in anchors:
    #print(link)
    if link.get("href").endswith("nc"):
        href = link.get("href")
        print(href)

oisst-avhrr-v02r01.20201201_preliminary.nc
oisst-avhrr-v02r01.20201202_preliminary.nc
oisst-avhrr-v02r01.20201203_preliminary.nc
oisst-avhrr-v02r01.20201204_preliminary.nc


In [33]:
# Find all the links in fetch_url which end with ".nc"
for link in soup.find_all('a'):
    
    # Find links that match update year
    if link.get('href').endswith(f'.nc'):
        
        # Get the link(s) that match
        href = link.get('href')
        #print(f"Download link match: {href}")
        
        # Use requests to download
        full_file = "/oisst-avhrr-v02r01."
        req_link = fetch_url + href
        print(req_link)
        req = requests.get(fetch_url + href)
        if req.raise_for_status():
            exit()
        
        
        
        ####. Status: At this point requests is able to connect with the different files
        # It would be desirable to use xarray to append them into a single month instead of daily files
        # BUT as single files its possible to keep track of the prelim data
        # OR if this is done daily and prelim files just get replaced, then you can just note that and save the month
        # The problem there is you need to replace the last two months until that 14 day period is done.
        
#         # Open link
        
#         file = open(cache_loc + href, 'wb')
#         chunk_size = 17000000
        
#         # Process in chunks
#         for chunk in req.iter_content(chunk_size):
#             file.write(chunk)
#         file.close()
#         print(f"Created {cache_loc} {href}")

https://www.ncei.noaa.gov/data/sea-surface-temperature-optimum-interpolation/v2.1/access/avhrr/202012/oisst-avhrr-v02r01.20201201_preliminary.nc
https://www.ncei.noaa.gov/data/sea-surface-temperature-optimum-interpolation/v2.1/access/avhrr/202012/oisst-avhrr-v02r01.20201202_preliminary.nc
https://www.ncei.noaa.gov/data/sea-surface-temperature-optimum-interpolation/v2.1/access/avhrr/202012/oisst-avhrr-v02r01.20201203_preliminary.nc
https://www.ncei.noaa.gov/data/sea-surface-temperature-optimum-interpolation/v2.1/access/avhrr/202012/oisst-avhrr-v02r01.20201204_preliminary.nc
